In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


Task 1:

Find the sales in terms of total dollars for all orders in each year, ordered from greatest to least. Do you notice any trends in the yearly sales totals?

In [6]:
# SQLite version

pd.read_sql_query(sql='''
SELECT strftime('%Y', occurred_at), SUM(total_amt_usd)
FROM orders
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,"strftime('%Y', occurred_at)",SUM(total_amt_usd)
0,2016,12864917.92
1,2015,5752004.94
2,2014,4069106.54
3,2013,377331.00
4,2017,78151.43


Solution:

 SELECT DATE_PART('year', occurred_at) ord_year,  SUM(total_amt_usd) total_spent
 FROM orders
 GROUP BY 1
 ORDER BY 2 DESC;

Task 2:

Which month did Parch & Posey have the greatest sales in terms of total dollars? Are all months evenly represented by the dataset?

In [7]:
# SQLite version

pd.read_sql_query(sql='''
SELECT strftime('%m', occurred_at), SUM(total_amt_usd), COUNT(*) count
FROM orders
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,"strftime('%m', occurred_at)",SUM(total_amt_usd),count
0,12,3129411.98,882
1,10,2427505.97,675
2,11,2390033.75,713
3,09,2017216.88,602
4,07,1978731.15,571
5,08,1918107.22,603
6,06,1871118.52,527
7,03,1659987.88,482
8,04,1562037.74,472
9,05,1537082.23,518


In [8]:
# SQLite version

pd.read_sql_query(sql='''
SELECT MIN(strftime('%Y-%m', occurred_at)) first_month,  MAX(strftime('%Y-%m', occurred_at)) last_month FROM
(SELECT strftime('%Y-%m', occurred_at), occurred_at
FROM orders
GROUP BY 1
ORDER BY 1);
''', con=conn)

,first_month,last_month
0,2013-12,2017-01


In [9]:
# SQLite version - how many months are present

pd.read_sql_query(sql='''
SELECT month, COUNT(*) count
FROM (SELECT strftime('%Y-%m', occurred_at), strftime('%m', occurred_at) month, SUM(total_amt_usd) total_usd
FROM orders
GROUP BY 1
ORDER BY 1) AS Tab1
GROUP BY 1
ORDER BY 1;
''', con=conn)

,month,count
0,01,4
1,02,3
2,03,3
3,04,3
4,05,3
5,06,3
6,07,3
7,08,3
8,09,3
9,10,3


In [10]:
# SQLite version - fair approach

pd.read_sql_query(sql='''
SELECT strftime('%m', occurred_at), SUM(total_amt_usd), COUNT(*) count
FROM orders
WHERE occurred_at BETWEEN '2014-01-01' AND '2017-01-01'
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,"strftime('%m', occurred_at)",SUM(total_amt_usd),count
0,12,2752080.98,783
1,10,2427505.97,675
2,11,2390033.75,713
3,09,2017216.88,602
4,07,1978731.15,571
5,08,1918107.22,603
6,06,1871118.52,527
7,03,1659987.88,482
8,04,1562037.74,472
9,05,1537082.23,518


Solution:

SELECT DATE_PART('month', occurred_at) ord_month, SUM(total_amt_usd) total_spent
FROM orders
WHERE occurred_at BETWEEN '2014-01-01' AND '2017-01-01'
GROUP BY 1
ORDER BY 2 DESC; 

Task 3:

Which year did Parch & Posey have the greatest sales in terms of total number of orders? Are all years evenly represented by the dataset?

In [11]:
# SQLite version

pd.read_sql_query(sql='''
SELECT strftime('%Y', occurred_at) ord_year, COUNT (*) total_sales
FROM orders
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,ord_year,total_sales
0,2016,3757
1,2015,1725
2,2014,1306
3,2013,99
4,2017,25


In [12]:
# SQLite version - how many months in every year

pd.read_sql_query(sql='''
SELECT strftime('%Y', occurred_at) month, COUNT(*) count
FROM (SELECT strftime('%Y-%m', occurred_at), occurred_at
FROM orders
GROUP BY 1
ORDER BY 1)
GROUP BY 1
ORDER BY 1;
''', con=conn)

,month,count
0,2013,1
1,2014,12
2,2015,12
3,2016,12
4,2017,1


Solution:

SELECT DATE_PART('year', occurred_at) ord_year,  COUNT(*) total_sales
FROM orders
GROUP BY 1
ORDER BY 2 DESC;

Task 4:

Which month did Parch & Posey have the greatest sales in terms of total number of orders? Are all months evenly represented by the dataset?

In [13]:
# SQLite version

pd.read_sql_query(sql='''
SELECT strftime('%m', occurred_at) ord_month, COUNT (*) total_sales
FROM orders
WHERE occurred_at BETWEEN '2014-01-01' AND '2017-01-01'
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,ord_month,total_sales
0,12,783
1,11,713
2,10,675
3,08,603
4,09,602
5,07,571
6,06,527
7,05,518
8,03,482
9,04,472


In [14]:
# SQLite version - how many months are present

pd.read_sql_query(sql='''
SELECT strftime('%m', occurred_at) month, COUNT(*) count
FROM (SELECT strftime('%Y-%m', occurred_at), occurred_at
FROM orders
GROUP BY 1
ORDER BY 1)
GROUP BY 1
ORDER BY 1;
''', con=conn)

,month,count
0,01,4
1,02,3
2,03,3
3,04,3
4,05,3
5,06,3
6,07,3
7,08,3
8,09,3
9,10,3


Solutions:

SELECT DATE_PART('month', occurred_at) ord_month, COUNT(*) total_sales
FROM orders
WHERE occurred_at BETWEEN '2014-01-01' AND '2017-01-01'
GROUP BY 1
ORDER BY 2 DESC;

Task 5:
In which month of which year did Walmart spend the most on gloss paper in terms of dollars?

In [15]:
# SQLite version

pd.read_sql_query(sql='''
SELECT strftime('%Y-%m', o.occurred_at) ord_date, SUM(o.gloss_amt_usd) gloss_usd
FROM orders o
JOIN accounts a
ON o.account_id = a.id
WHERE a.name = 'Walmart'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;
''', con=conn)

,ord_date,gloss_usd
0,2016-05,9257.64


Solution:

SELECT DATE_TRUNC('month', o.occurred_at) ord_date, SUM(o.gloss_amt_usd) tot_spent
FROM orders o 
JOIN accounts a
ON a.id = o.account_id
WHERE a.name = 'Walmart'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;